<a href="https://colab.research.google.com/github/Exferro/anqs_quantum_chemistry/blob/main/colab_toy_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To download the package source code from GitHub, uncomment and execute the following cell:

In [ ]:
#!git clone https://github.com/Exferro/anqs_quantum_chemistry.git

To install the required packages, uncomment and execute the following cell:

In [ ]:
#!pip install -r ./anqs_quantum_chemistry/requirements_colab.txt

In [ ]:
import os
import json

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [ ]:
import sys
import importlib

To install our package, uncomment and execute the following cell:

In [ ]:
#!cd ./anqs_quantum_chemistry/nqs; python -m pip install -e .

Add the path to our package to Colab PATH variable.

In [ ]:
sys.path.append('./anqs_quantum_chemistry/nqs')

Here we import the required classes from our package. Sometimes imports might not work immediately after the installation of `nqs` and in this situation we would advise to simply restart the Colab runtime.

In [ ]:
from nqs.applications.quantum_chemistry.molecule import GeometryConfig, MolConfig
from nqs.applications.quantum_chemistry.experiments.preparation import create_mol

from nqs.applications.quantum_chemistry.experiments.energy_opt_exp import EnergyOptExpConfig, EnergyOptExp

from nqs.applications.quantum_chemistry import CHEMICAL_ACCURACY

In [ ]:
mols_root_dir = './molecules'

mol_name = 'Li2O'

We specify the molecule geometry and save it as a .json file which will be later used to construct the molecule object.

In [ ]:
geom = [["O", [0.0, 0.0, 0.0]],
        ["Li", [0.0, 0.0, 1.51903]],
        ["Li", [0.0, 0.0, -1.51903]]]

geom_config = GeometryConfig(type='toy', idx=0)
geom_dir = os.path.join(mols_root_dir,
                       f'name={mol_name}',
                       'geometries',
                        geom_config.to_path_suffix())

if not os.path.exists(geom_dir):
    os.makedirs(geom_dir)

geom_filename = os.path.join(geom_dir, 'geom.json')

if not os.path.exists(geom_filename):
    with open(geom_filename, 'w') as f:
        json.dump(geom, f)
else:
    print(f'You are trying to overwrite an existing geometry at {geom_filename}')

We create a molecule object, obtain the second quantised Hamiltonian for it and run several quantum chemistry methods to compare to. It takes ~5 minutes to run those calculations on Colab. However, once they are done, we cache them, and thus subsequent calls to `create_mol` should be much faster.

In [ ]:
mol_config = MolConfig(name=mol_name,
                       geom_config=geom_config,
                       basis='sto-3g')
mol = create_mol(config=mol_config,
                 mols_root_dir=mols_root_dir)

We use a high-level wrappers around elementary ANQS subroutines, which would set up the directory tree for us, and create the required objects (Hilbert space, Hamiltonian, ansatz) for us with the default values.

In [ ]:
series_name = 'toy_example'
exp_config = EnergyOptExpConfig(mols_root_dir=mols_root_dir,
                                mol_config=mol_config,
                                series_name=series_name)
exp_config.local_energy_config.use_tree_for_candidates = 'trie'
exp_config.proc_grad_schedule[0][1].use_sr = True
exp_config.proc_grad_schedule[0][1].sr_config.max_indices_num = 50

We create an instance of energy optimisation experiment, which contains everything a VMC experiment should: a loop to sample, evaluate amplitudes, the energy, the gradient, and stochastic reconfiguration.

In [ ]:
energy_opt_exp = EnergyOptExp(config=exp_config,
                              mol=mol)

We run the experiment! It takes ~3 minutes on Colab for 500 iterations.

In [ ]:
for method in ('hf', 'cisd', 'ccsd', 'ccsd_t', 'fci'):
    print(f'{method} energy: {getattr(energy_opt_exp.mol, f"{method}_energy")}')
if energy_opt_exp.mol.fci_energy is not None:
    print(f'fci energy up to chem. acc.: {energy_opt_exp.mol.fci_energy + CHEMICAL_ACCURACY}')
energy_opt_exp.run(iter_num=500)

# Building plots

We need to extract the automatically saved results from the tree of directories.

In [ ]:
series_dir = os.path.join(mol.dir, 'exp_series', series_name)
result_filename = os.path.join(series_dir, f'{exp_config.to_sha256_str()}', 'result.csv')
result = pd.read_csv(result_filename)

We plot how the ANQS energy changes in the course of optimisation and compare it to the conventional quantum chemistry methods. The grey shaded area corresponds to chemical accuracy.

In [ ]:
fig, ax = plt.subplots()
ax.grid()
ax.plot(result['iter_idx'],
        result['sample_aware_energy'] - mol.fci_energy,
        label='ANQS')
ax.set_yscale('log')

x_lim = ax.get_xlim()

point_num = 10
x_axis = np.linspace(x_lim[0], x_lim[1], point_num)
for method in ('hf', 'cisd', 'ccsd', 'ccsd_t'):
    ax.plot(x_axis,
            np.ones_like(x_axis) * getattr(mol, f'{method}_energy') - mol.fci_energy,
            label=method,
            linestyle='dashed',
            linewidth=1)
ax.fill_between([x_lim[0], x_lim[1]], 0, 1.6e-3, color='grey', alpha=0.15)
ax.set_xlim(x_lim)

ax.set_xlabel('Iteration')
ax.set_ylabel('Difference with FCI [Ha]')
ax.legend(loc='best')